In [14]:
# from google.colab import drive
# drive.mount('/content/drive')


In [1]:
import requests
from urllib.parse import urlencode, quote_plus
import numpy as np
import sys
from dotenv import find_dotenv, load_dotenv
import os
import pandas as pd

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sayemkamal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sayemkamal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sayemkamal/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# **Citing this code**
This code is the second version of a Expertise finding tool developed by Volz et al. 2023 (https://ui.adsabs.harvard.edu/abs/2023AAS...24210207V/abstract).<br>
It utilizes NASA ADS API to query for articles (refereed or not) in the "Astronomy" database (cite ADS).
Please, cite "Helfenbein et al. 2023 (in prep) and refer to the README file in the github.

**Directory set up**<br>
The file *stopwords.txt* is utilized to create meaningful N-grams. Make sure to provide an accurate path in the following cell.<br> Also, the path will be used by the code in other instances to identify other useful files.

In [2]:
path_stop= '/Users/sayemkamal/NASA_Project/ReviewerExtractor/codeV3/'
stop_file='stopwords.txt'
stop_dir=path_stop+stop_file
sys.path.append(path_stop)



In [3]:
#token = 'Your own token from ADS API page ' #Insert your API token
token = 'WIhvOVWrt4ksJRFq7P0fuYToniASsS5Wlp5NC28V' 


In [4]:
#For the TextAnalysis File, please refer to M. Volze et al. 2023
import TextAnalysis as TA
import ADSsearcherpkg as AP


# **Example 1: Searching expertises of a single person or institution based on their name**

#### The following examples will work with the **ads_search** function. These are the parameters of ads_search:

**name:** The name of the desired author (formatted "Last,First")  
**institution:** The name of the desired institution  
**year:** The year or range of years (formatted '[YEAR1 TO YEAR2]') of the desired publications  
**refereed:** Parameter that includes only refereed paper adding (formatted "property:refereed")  
**token:** API key for ADS  
**stop_dir:** Directory for list of stopwords for n-grams analysis  
**second_auth:** Boolean parameter that determines whether second author publications are included  
**groq_analysis:** Boolean parameter that determines whether groq expertise analysis will be run.


In [6]:
datf=AP.ads_search(name="Cucchiara, Antonino",
               token=token, stop_dir=stop_dir,groq_analysis=False)


I will search for papers matching the following criteria:
author:"^Cucchiara, Antonino"

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 1 rows in parallel with max_workers=2...
Raw response for row 0: ["gamma ray burst:GRB", "cosmology:dark energy", "stars:star formation"]
Groq analysis complete.


/Users/sayemkamal/NASA_Project/ReviewerExtractor/codeV3/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [7]:
datf


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Subtopics
0,"Cucchiara, Antonino","None, None, None, None, None, None, None, None...","Cucchiara, Antonino, Cucchiara, Antonino, Cucc...","2024AAS...24324404C, 2024AAS...24314802C, 2024...","[NASA NHFP in the next decade, The Logistics f...","2024-02-00, 2024-02-00, 2024-02-00, 2023-06-00...","[Gamma-ray bursts, Interstellar medium, Interg...","NASA HQ, NASA HQ, NASA Headquarters, NASA HQ, ...",The NASA Hubble Fellowship Program (NHFP) has ...,"[2024AAS...24324404C, 2024AAS...24314802C, 202...","[(grb, 94), (grbs, 37), (redshift, 32), (after...","[(al gcn, 18), (gamma ray, 17), (ray burst, 15...","[(gamma ray burst, 8), (gamma ray bursts, 7), ...",Dirty,"[gamma ray burst:GRB, cosmology:dark energy, s..."


### **Example 1a: Single Name Search, including second author publications**

The second_auth parameter allows for searches that include publications in which the author was a first or second author.

In [ ]:
datf2=AP.ads_search(institution="University of California Berkeley",
               token=token, stop_dir=stop_dir, second_auth=True)


In [ ]:
datf


### **Example 1b: Single Name with LLM subtopics analysis**

In [ ]:
datf=AP.ads_search(name="Cucchiara, Antonino", token=token, stop_dir=stop_dir,groq_analysis=True)


In [ ]:
datf


In [12]:
print(datf['Subtopics'][0])


['gamma ray burst:GRB', 'cosmology:dark energy', 'stars:star formation']


# **Example 2: Searching expertises of ALL scientists that published as first authors when affiliated to single institution name**

The search will focus on papers and all authors that have published in the past 15 years at a specific institution (academic or otherwise): <br>
The format for a single institution is as follows: **institution="Institution Name"**. <br>
**Caveat**: It is possible that the institutions as input by the user does not match what has been cataloged in ADS, therefore if the final output is empty, make sure to try different versions of the institution names (e.g. Cal Poly Pomona, Cal Poly, California Polytechnic State University) to get the most complete list of authors.

In [7]:
datf=AP.ads_search(institution="Hampton University",refereed="property:refereed",
               token=token, stop_dir=stop_dir,groq_analysis=False)


I will search for papers matching the following criteria:
pos(institution:"Hampton University",1)

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 41 rows in parallel with max_workers=2...
Raw response for row 1: ["physical data and processes:magnetic fields", "physical data and processes:plasmas", "physical data and processes:instabilities"]
Raw response for row 0: ["physical data and processes: radiation mechanisms: general", "physical data and processes: radiation mechanisms: non-thermal", "physical data and processes: radiation mechanisms: thermal"]
Raw response for row 2: ["the sun:Sun:atmosphere", "physical data and processes:radiation mechanisms:general"]
Raw response for row 3: ["physical data and processes: radiation mechanisms: general", "the sun: Sun: activity", "the sun: Sun: atmosphere"]


In [23]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Afanasev, A.",Hampton University,"Afanasev, A.",2008PhRvL.101l0401A,[Experimental Limit on Optical-Photon Coupling...,2008-09-00,"[12.20.Fv, 11.30.Ly, 12.60.Cn, 13.40.Hq, Exper...","Department of Physics, Hampton University, Ham...",We report on the first results of a sensitive ...,"[10.48550/arXiv.0806.2631, arXiv:0806.2631, 20...","[(light, 4), (coupling, 3), (photon, 3), (resu...","[((scalar, coupling), 2), ((laser, light), 2),...","[((report, results, sensitive), 1), ((results,...",Clean
1,"Ali, Halima","Hampton University, Hampton University","Ali, Halima, Ali, Halima","2011JPlPh..77..785A, 2009JPlPh..75..303A",[Symplectic calculation of the outboard magnet...,"2011-12-00, 2009-06-00",[],"Hampton University, Hampton, VA 23668, USA ( )...",The backward symplectic DIII-D map and continu...,"[10.1017/S0022377811000213, 2011JPlPh..77..785...","[(magnetic, 10), (twm, 10), (perturbation, 9),...","[((field, line), 4), ((flux, preserving), 4), ...","[((flux, preserving, twm), 4), ((field, line, ...",Dirty
2,"Bailey, S. M.",Hampton University,"Bailey, S. M.",1999SoPh..186..243B,[Sounding rocket measurements of the solar sof...,1999-05-00,"[Silicon, Thin Film, Active Area, Reasonable A...","Center for Atmospheric Science, Hampton Univer...",Measurements of the solar soft X-ray (XUV: 2 n...,"[10.1023/A:1005141503558, 1999SoPh..186..243B]","[(solar, 4), (spectrum, 3), (measurement, 2), ...","[((solar, spectrum), 3), ((soft, ray), 2), ((m...","[((model, solar, spectra), 2), ((measurements,...",Dirty
3,"Bailey, Scott M.",Hampton University,"Bailey, Scott M.",1999GeoRL..26.1255B,[Measurements of the solar soft X-ray irradian...,1999-05-00,[Atmospheric Composition and Structure: Thermo...,"Center for Atmospheric Sciences, Hampton Unive...","Beginning on March 11, 1998, the Student Nitri...","[10.1029/1999GL900236, 1999GeoRL..26.1255B]","[(solar, 6), (ray, 3), (irradiances, 3), (cm, ...","[((soft, ray), 2), ((ray, irradiance), 2), ((s...","[((soft, ray, irradiance), 2), ((beginning, ma...",Dirty
4,"Baraka, Suleiman M.",Hampton University,"Baraka, Suleiman M.",2021JGRA..12629528B,[The Impact of Radial and Non-Radial IMF on th...,2021-10-00,"[particle-in-cell, solar wind magnetosphere co...","National Institute of Aerospace, Hampton, VA U...",The boundary between the solar wind and the Ea...,"[2021JGRA..12629528B, 10.1002/essoar.10506965....","[(radial, 6), (mp, 5), (imf, 5), (backstreamin...","[((radial, imf), 4), ((radial, radial), 2), ((...","[((radial, radial, imf), 2), ((boundary, solar...",Dirty
5,"Basu Sarkar, Debajyoti",Hampton University,"Basu Sarkar, Debajyoti",2022GeoRL..4900987B,[Influence of Planetary Surface Temperature on...,2022-11-00,[],Department of Atmospheric and Planetary Scienc...,Heat transport from the interior to the surfac...,"[10.1029/2022GL100987, 2022GeoRL..4900987B]","[(heat, 6), (surface, 4), (terrestrial, 4), (p...","[((heat, pipe), 4), ((heat, transport), 2), ((...","[((heat, pipe, volcanism), 2), ((heat, transpo...",Dirty
6,"Bell, Jared M.",Hampton University,"Bell, Jared M.",2014JGRA..119.4957B,[Developing a self-consistent description of T...,2014-06-00,"[Titan, thermosphere, atmospheric escape, Cass...",Center for Planetary Atmospheres and Flight Sc...,"In this study, we develop a best fit descripti...","[10.1002/2014JA019781, 2014JGRA..119.4957B]","[(escape, 5), (titan, 4), (atmospheric, 4), (u...","[((titan, upper), 3), ((upper, atmosphere), 3)...","[((titan, upper, atmosphere), 3), ((study, dev...",Dirty
7,"Blalock, John J. J.",Hampton University,"Blalock, John J. J.",2019PhDT.......207B,[Measuring seasonal evolution of dynamics and ...,2019-00-00,[],"Hampton University, United States",,[2019PhDT.......207B],[],[],[],Dirty
8,"Box, M. A.","Hampton University, Hampton University","Box, M. A., Box, M. A.","1981JAtS...38.1037B, 1979A

# **Example 3: Searching a single author publication while affiliated to a specific institution**

The search will focus on papers published by a single author while they are affiliated to a specific institution, in the past 15 years:<br>

The format for a single author and institution is as follows: **name= 'Last, First', institution= 'Institution Name'**.

In [24]:
datf=AP.ads_search(name= 'Capper, Daniel', institution="University of Southern Mississippi",
               token=token, stop_dir=stop_dir)


I will search for papers matching the following criteria:
author:"^Capper, Daniel" AND pos(institution:"University of Southern Mississippi",1)

I am now querying ADS.



In [25]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Capper, Daniel","University of Southern Mississippi, University...","Capper, Daniel, Capper, Daniel","2022SpPol..5901462C, 2019SpPol..4901325C",[What Should We Do with Our Moon?: Ethics and ...,"2022-02-00, 2019-08-00","[Environmental justice, Environmental preserva...","School of Humanities, University of Southern M...",The planned expansion of the human presence on...,"[10.1016/j.spacepol.2021.101462, 2022SpPol..59...","[(environmental, 8), (human, 7), (moon, 7), (r...","[((baseline, ecology), 3), ((environmental, sc...","[((baseline, ecology, method), 2), ((planned, ...",Dirty


# **Example 4: Searching a single author name within a different time-frame**

The search will focus on papers from one single author that were published in a different time-frame. There are two options for doing so:
   - A single year (e.g. 2010): in this case the code will query ADS for articles published by the specified authors between one year prior to 4  years after. So searching year='2010' will search articles between 2009 and 2014<br>
   - A year range: in this case the syntax is year='[YEAR TO YEAR]' (e.g. year='[2009 TO 2023]') <br>

The format for a single author name remains the same as before: **name= 'Last, First'**. <br>

Here are two examples:
- Searching for Dr. Pepper's articles between year 1999 and 2004
- Searching for Dr. Pepper's articles between year 2019 and 2023

In [26]:
datf=AP.ads_search(name= 'Pepper, Joshua', year='2000',
               token=token, stop_dir=stop_dir)


I will search for papers matching the following criteria:
author:"^Pepper, Joshua" AND pubdate:[1999 TO 2004]

I am now querying ADS.



In [27]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Pepper, Joshua","None, None, None, None, None","Pepper, J., Pepper, Joshua, Pepper, J., Pepper...","2004AAS...20517102P, 2004AIPC..713..185P, 2003...","[Deployment and Initial Data from KELT, KELT: ...","2004-12-00, 2004-06-00, 2003-09-00, 2002-12-00...","[97.82.Fs, 97.82.Cp, Substellar companions, pl...","OSU, Department of Astronomy, 4055 McPherson L...",The Kilodegree Extremely Little Telescope (KEL...,"[2004AAS...20517102P, 2005BAAS...37..375P, 200...","[(star, 15), (transit, 13), (telescope, 12), (...","[((bright, star), 9), ((transit, bright), 8), ...","[((transits, bright, stars), 8), ((bright, sta...",Dirty


In [5]:
datf=AP.ads_search(name= 'Pepper, Joshua', year='[2019 TO 2023]',
               token=token, stop_dir=stop_dir)


I will search for papers matching the following criteria:
author:"^Pepper, Joshua" AND pubdate:[2019 TO 2023]

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 1 rows in parallel with max_workers=2...
Raw response for row 0: ["stars:stellar evolution", "planetary systems:detection", "planetary systems:formation"]
Groq analysis complete.


/Users/sayemkamal/NASA_Project/ReviewerExtractor/codeV3/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [29]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Pepper, Joshua","None, None, None, None, None, None, None","Pepper, Joshua, Pepper, Joshua, Pepper, Joshua...","2021plat.confE..96P, 2021tsc2.confE.139P, 2020...","[The TESS Input Catalog and lessons for PLATO,...","2021-10-00, 2021-07-00, 2020-06-00, 2020-01-00...","[Input catalogue, TESS, Zenodo community plato...","Lehigh University, Lehigh University, Departme...",The presentation compares the input catalogues...,"[2021plat.confE..96P, 10.5281/zenodo.5577992, ...","[(star, 14), (tess, 12), (publication, 10), (t...","[((host, star), 5), ((publication, record), 3)...","[((presentation, compares, input), 1), ((compa...",Clean


In [30]:
datf=AP.ads_search(name= 'Cucchiara, Antonino', institution="University of California, Berkeley",token=token,stop_dir=stop_dir, year='[2010 TO 2014]')


I will search for papers matching the following criteria:
author:"^Cucchiara, Antonino" AND pos(institution:"University of California, Berkeley",1) AND pubdate:[2010 TO 2014]

I am now querying ADS.

DataFrame is empty! Trying affiliation instead of institution.
Trying alternative search: pos(aff:"University of California, Berkeley",1) AND pubdate:[2010 TO 2014]
No results found.


In [31]:
datf


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Data Type,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams
0,"Cucchiara, Antonino","University of California, Berkeley",None,None,None,None,None,None,None,None,None,None,None,None


# **Example 5: Searching a single institution name within a specific time-frame**

The search will focus on authors that publishes as first authors affiliated to a specific institution in a defined timespan. <br>
The format for a author name is the same in previous example (**"Last name, First name"**) and specified year range is similar to the option provided earlier:<br>
   - A single year (e.g. 2010): in this case the code will query ADS for articles published by the specified authors between one year prior to 4  years after. So searching year='2010' will search articles between 2009 and 2014<br>
   - A year range: in this case the syntax is year='[YEAR TO YEAR]' (e.g. year='[2009 TO 2023]') <br>

Following we present two examples:

In [32]:
datf=AP.ads_search(institution="University of Southern Mississippi",year='2000',
               token=token, stop_dir=stop_dir)


I will search for papers matching the following criteria:
pos(institution:"University of Southern Mississippi",1) AND pubdate:[1999 TO 2004]

I am now querying ADS.



In [33]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Buchanan, Randy K.",University of Southern Mississippi,"Buchanan, Randy K.",2003AIPC..684.1121B,[Simulating Martian Temperatures],2003-09-00,"[07.20.Mc, 95.55.-n, Cryogenics, refrigerators...","Electronic Engineering Technology, University ...",The Mars Electrostatic Chamber (MEC) was desig...,"[2003AIPC..684.1121B, 10.1063/1.1627280]","[(temperature, 9), (control, 5), (mar, 4), (ch...","[((temperature, control), 3), ((pressure, atmo...","[((mars, electrostatic, chamber), 1), ((electr...",Dirty


# **Example 6: Searching a single Author, at a specific institution and within a specific time-frame**

The following example combines several of the previous ones in a single search.
Specifically:<br>
   - A single author<br>
   - Affiliated to a single institutions<br>
   - In a specific time frame of publications<br>
    
Please, refer to the previous examples for the sintax required. <br>
Here are an example

In [34]:
datf=AP.ads_search(name= 'Brown, Beth A.', institution="Howard university",year='[2009 TO 2022]',
               token=token, stop_dir=stop_dir)


I will search for papers matching the following criteria:
author:"^Brown, Beth A." AND pos(institution:"Howard university",1) AND pubdate:[2009 TO 2022]

I am now querying ADS.



In [35]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Brown, Beth A.",Howard university,"Brown, Beth A.",2010AIPC.1280....3B,[Causal Structures of Dynamic Black Holes],2010-10-00,"[black holes, space-time configurations, satel...","NASA Goddard Space Flight Center, Greenbelt, M...","Dynamic space-times, especially those manifest...","[10.1063/1.3507200, arXiv:0904.4192, 2010AIPC....","[(black, 3), (hole, 3), (dynamic, 2), (behavio...","[((black, hole), 3), ((hole, presented), 2), (...","[((black, hole, presented), 2), ((dynamic, spa...",Dirty


# **Example 7: Searching through a list of institutions**

The search will focus on papers from a list of institutions, so the input is a csv file that has multiple institution names stored in it. This will then find all papers from those institutions (**see CAVEATS in Example 2 above related to Institution Names)**:<br>

The input file has to be a .csv file (e.g."top10inst.csv"), and must contain at least one column titled  **"Current Institution"** or **"Institution"** (the first cell of the column is usually interpreted as such). The file can contains other columns, they will be ignored.<br>
If the file is in a different directory than the one where the code it, include the whole path. <br>

The code will run as in Example 2 above for each institutions and append the results at each iteration providing a final dataframe with all the researchers at all the institutions in the list provided.<br>
**NOTE: at the moment if an institution query returns an empty dataframe the code will ignore it and continue to the following one.**


In [6]:
datf=AP.run_file_search(filename='NSF_Fellows_expertiseFeb52025.csv',
               token=token, stop_dir=stop_dir)



What type of search do you want to conduct?
-Enter 'name' for Name Search - search by author name
-Enter 'institution' for Institution Search - search by institution
-Enter 'fellow' for Fellow Search - search by name, institution, and year


You are running 'name' search.

Listed are the available columns from your dataset: Unnamed: 0, Input Author, Input Institution, First Author, Bibcode, Title, Publication Date, Keywords, Affiliations, Abstract, Identifier, Top 10 Words, Top 10 Bigrams, Top 10 Trigrams, Data Type
Searching for results...
I will search for papers matching the following criteria:
author:"^Appel, Sabrina" AND pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 1 searches - Processed name: Appel, Sabrina (including only first author)
I will search for papers matching the following criteria:
author:"^Blunt, Sarah" AND pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 2 searches - Processed name: Blunt, Sarah (including only first author)
I will search for papers matching the following criteria:
author:"^Cortes, Jorge" AND pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 3 searches - Processed name: Cortes, Jorge (including only first author)
I will search for papers matching the follow

In [37]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Browning, Matthew","None, None, None, None, None, None, None, None...","Browning, M. K., Browning, Matthew K., Brownin...","2021csss.confE..80B, 2020mdps.conf..141B, 2019...",[Modelling X-ray and radio emission from a fla...,"2021-03-00, 2020-01-00, 2019-05-00, 2017-05-00...","[Young stars, Flares, Radio emission, Astrophy...","Uni of Manchester, -, -, University of Exeter,...",T-Tauri stars exhibit strong flaring activity....,"[2021csss.confE..80B, 10.5281/zenodo.4562731, ...","[(field, 88), (magnetic, 66), (rotation, 64), ...","[((magnetic, field), 42), ((differential, rota...","[((magnetic, dynamo, action), 9), ((low, mass,...",Dirty
1,"Cruz, Kelle","None, None, None, None, None, None, None, None...","Cruz, Kelle, Cruz, Kelle, Cruz, Kelle, Cruz, K...","2024ESS.....562615C, 2024AAS...24320219C, 2021...",[The SIMPLE Archive: A collaboratively-curated...,"2024-04-00, 2024-02-00, 2021-03-00, 2018-09-00...","[Very low mass stars, Astronomy, archives, Sta...","CUNY Hunter College &amp; AMNH, CUNY Hunter Co...",We present the SIMPLE Archive of low mass star...,"[2024ESS.....562615C, 2024AAS...24320219C, 10....","[(dwarf, 122), (mass, 46), (brown, 46), (low, ...","[((brown, dwarf), 46), ((ultracool, dwarf), 20...","[((stars, brown, dwarfs), 11), ((low, mass, st...",Dirty
2,"Gawiser, Eric","None, None, None, None, None, None, None, None...","Gawiser, Eric, Gawiser, Eric, Gawiser, Eric, G...","2024APS..APRB03008G, 2024AAS...24321105G, 2024...",[Improving 3x2pt Cosmology Constraints: Traini...,"2024-04-00, 2024-02-00, 2024-02-00, 2023-01-00...","[98.62.Ai, 98.62.Lv, 98.58.Hf, 98.54.Ep, 98.54...","Rutgers University, Rutgers University, Rutger...",Large imaging surveys of galaxies rely on phot...,"[2024APS..APRB03008G, 2024AAS...24321105G, 202...","[(galaxy, 171), (formation, 75), (star, 67), (...","[((star, formation), 57), ((lyman, alpha), 39)...","[((star, formation, rate), 18), ((star, format...",Clean


# **Example 8: Searching through a list of Authors names**

The search will focus on papers from a list of authors names (similar format as Example 1 above, **'Last, First'**). <br>
The input is a .csv file that has multiple authors names stored in it under a column Title: **"Name"**. <br>
The ADS search will focus on the period 2003 to 2023.
<br>
If the file is in a different directory than the one where the code it, include the whole path. <br>

The code will then execute the search one name after the other and uppend each result to the previous one.<br>
In the following example we use, for convenience, the same example file as before which also contain a list of researchers names.


In [5]:
datf=AP.run_file_search(filename='Fellowships_updated.csv',
               token=token, stop_dir=stop_dir)



What type of search do you want to conduct?
-Enter 'name' for Name Search - search by author name
-Enter 'institution' for Institution Search - search by institution
You are running 'name' search.

Listed are the available columns from your dataset: Full Name
Searching for results...
I will search for papers matching the following criteria:
(author:"^Graur, Or" OR pos(author:"Graur, Or",2)) AND pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 1 searches - Processed name: Graur, Or (including second author)
I will search for papers matching the following criteria:
(author:"^Kamenetzky, Julia" OR pos(author:"Kamenetzky, Julia",2)) AND pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 2 searches - Processed name: Kamenetzky, Julia (including second author)
I will search for papers matching the following criteria:
(author:"^Barger, Kat" OR pos(author:"Barger, Kat",2)) AND pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 3 searches - Processed name: Barger, Kat (i

In [5]:
datf=AP.run_file_search(filename='Carn25_R1.csv',
               token=token, stop_dir=stop_dir)



What type of search do you want to conduct?
-Enter 'name' for Name Search - search by author name
-Enter 'institution' for Institution Search - search by institution


You are running 'institution' search.

Listed are the available columns from your dataset: unitid, name, city, state, level, control, Undergraduate Program, Graduate Program, Enrollment Profile, Undergraduate Profile, Size & Setting, Basic, Community Engagement, Leadership for Public Practice, 2025 Research Activity Designation, 187 results for  all categories
Searching for results...
Processing institution: American University
I will search for papers matching the following criteria:
pos(institution:"American University",1) AND pubdate:[2005 TO 2030]

I am now querying ADS.

Processing institution: Arizona State University
I will search for papers matching the following criteria:
pos(institution:"Arizona State University",1) AND pubdate:[2005 TO 2030]

I am now querying ADS.

Processing institution: Auburn University
I will search for papers matching the following criteria:
pos(institution:"Auburn University",1) AND pubdate:[2005 TO 2030]

I am now querying ADS.

Processing institutio

In [10]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")
datf.to_excel('CarnR1_Output.xlsx', index=False, engine='openpyxl')


In [40]:
print(len(datf.iloc[0]['Title']))


66
